In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding

# 📌 Step 1: Define Dataset
data = {
    "conversations": [
        ["Hi", "Hello! How can I assist you today?"],
        ["What's your favorite color?", "I don't have favorites, but I think blue is quite popular!"],
        ["Who invented the light bulb?", "Thomas Edison is credited with inventing the light bulb."],
        ["Tell me a riddle", "Sure! What has to be broken before you can use it? An egg!"],
        ["What is AI?", "AI, or Artificial Intelligence, refers to machines that can simulate human thinking."],
        ["Who is the president of the USA?", "You can check the latest updates online!"],
        ["Tell me a movie recommendation", "If you like sci-fi, try 'Inception' or 'Interstellar'."],
        ["How do I stay productive?", "Try setting goals, taking breaks, and prioritizing important tasks!"],
        ["What is the capital of Japan?", "The capital of Japan is Tokyo."],
        ["What is 2 + 2?", "2 + 2 equals 4!"],
        ["Do you like music?", "I don't listen to music, but I can suggest some songs if you'd like!"],
        ["Goodbye", "Goodbye! Have a great day!"]
    ]
}

# 📌 Step 2: Preprocess Dataset
questions = []
answers = []

for pair in data["conversations"]:
    questions.append(pair[0])
    answers.append("<start> " + pair[1] + " <end>")  # Special tokens for training

# Ensure special tokens are included in tokenization
special_tokens = ["<start>", "<end>"]
tokenizer = Tokenizer(filters='')  # Prevents removal of special characters
tokenizer.fit_on_texts(special_tokens + questions + answers)

vocab_size = len(tokenizer.word_index) + 1

question_sequences = tokenizer.texts_to_sequences(questions)
answer_sequences = tokenizer.texts_to_sequences(answers)

max_length = max([len(seq) for seq in question_sequences + answer_sequences])

question_padded = pad_sequences(question_sequences, maxlen=max_length, padding='post')
answer_padded = pad_sequences(answer_sequences, maxlen=max_length, padding='post')

# 📌 Step 3: Build LSTM Model
embedding_dim = 100
lstm_units = 256

# Encoder
encoder_inputs = Input(shape=(max_length,))
encoder_embedding = Embedding(vocab_size, embedding_dim, mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(lstm_units, return_state=True)
_, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(max_length,))
decoder_embedding = Embedding(vocab_size, embedding_dim, mask_zero=True)(decoder_inputs)
decoder_lstm = LSTM(lstm_units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# 📌 Step 4: Training Preparation
decoder_target_data = np.zeros_like(answer_padded)
decoder_target_data[:, :-1] = answer_padded[:, 1:]

# Model Training
model.fit(
    [question_padded, answer_padded], decoder_target_data,
    batch_size=32, epochs=200, verbose=1
)

model.save("chatbot_model.h5")
print("✅ Model Training Completed & Saved")

# 📌 Step 5: Define Separate Inference Models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(lstm_units,))
decoder_state_input_c = Input(shape=(lstm_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_embedding2 = Embedding(vocab_size, embedding_dim, mask_zero=True)(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(decoder_embedding2, initial_state=decoder_states_inputs)
decoder_states = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states
)

# 📌 Step 6: Greedy Search Response Generation
def predict_response(input_text):
    input_seq = tokenizer.texts_to_sequences([input_text])
    input_seq = pad_sequences(input_seq, maxlen=max_length, padding='post')

    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = tokenizer.word_index.get('<start>', 1)

    response = []
    for _ in range(max_length):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        token_idx = np.argmax(output_tokens[0, -1, :])
        word = tokenizer.index_word.get(token_idx, '')

        if word == '<end>' or word == '':
            break

        response.append(word)

        target_seq[0, 0] = token_idx
        states_value = [h, c]

    return ' '.join(response)

# 📌 Step 7: Chat with Bot
print("\n🤖 Chatbot is ready! Type 'exit' to stop chatting.")
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit', 'bye']:
        print("Chatbot: Goodbye!")
        break
    response = predict_response(user_input)
    print(f"Chatbot: {response}")


Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - loss: 4.6832
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step - loss: 4.6749
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 305ms/step - loss: 4.6662
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 294ms/step - loss: 4.6565
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step - loss: 4.6449
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step - loss: 4.6303
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step - loss: 4.6105
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 307ms/step - loss: 4.5821
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 313ms/step - loss: 4.5384
Epoch 10/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 299ms/step - loss: 4.4669
Epoch 11/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step - loss: 4.3490
Epoch 12/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 314ms/step - loss: 4.1897
Epoch 13/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 376ms/step - loss: 4.0894
Epoch 14/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step - loss: 4.1097
Epoch 15/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step - loss: 4.1168
Epoch 1

✅ Model Training Completed & Saved

🤖 Chatbot is ready! Type 'exit' to stop chatting.
You: Hi
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Chatbot: hello! how can can
You: Tell me a riddle
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Chatbot: sure! what has to be broken before you use it? an
You: What is AI?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━